Feature explanations at: https://www.nationalgrideso.com/data-portal/historic-demand-data/historic_demand_data_2023

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/external/demanddata.csv')

### Seasonal feature engineering

In [3]:
df['SETTLEMENT_DATE'] = pd.to_datetime(df['SETTLEMENT_DATE'])

df['month'] = df['SETTLEMENT_DATE'].dt.month
df['day_of_week'] = df['SETTLEMENT_DATE'].dt.dayofweek


# apply sinusoidal transformations
# why? -> removes ordinal relationships (months 1-12, days 1-7) while not creating many new features such as via one hot encoding
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week']/7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week']/7)

df['settlement_period_sin'] = np.sin(2 * np.pi * df['SETTLEMENT_PERIOD']/7)
df['settlement_period_cos'] = np.cos(2 * np.pi * df['SETTLEMENT_PERIOD']/7)

df.head()

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,NSL_FLOW,ELECLINK_FLOW,month,day_of_week,month_sin,month_cos,day_of_week_sin,day_of_week_cos,settlement_period_sin,settlement_period_cos
0,2023-01-01,1,21043,24680,18995,2815,6545,0,14284,0,...,235,998,1,6,0.5,0.866025,-0.781831,0.62349,0.781831,0.623490
1,2023-01-01,2,21756,25289,19730,2793,6545,0,14284,0,...,157,998,1,6,0.5,0.866025,-0.781831,0.62349,0.974928,-0.222521
2,2023-01-01,3,21348,25269,19327,2773,6545,0,14284,0,...,-219,999,1,6,0.5,0.866025,-0.781831,0.62349,0.433884,-0.900969
3,2023-01-01,4,20586,24574,18589,2753,6545,0,14284,0,...,-298,999,1,6,0.5,0.866025,-0.781831,0.62349,-0.433884,-0.900969
4,2023-01-01,5,19781,24135,17806,2712,6545,0,14284,0,...,-679,998,1,6,0.5,0.866025,-0.781831,0.62349,-0.974928,-0.222521


### Lag Features by 1 Day
We do this by incrementing the settlement-date column by 1, effectively converting all features to the observation 24 hours ago. 

In [4]:
df['SETTLEMENT_DATE'] = df['SETTLEMENT_DATE'] + pd.Timedelta(days=1)

### Reconstruct Datetime Column

In [5]:
from datetime import timedelta

# Adjust the calculation for hours and minutes
def calculate_datetime(row):
    minute_of_day = (row['SETTLEMENT_PERIOD'] - 1) * 30
    hour = minute_of_day // 60
    minute = minute_of_day % 60
    datetime = row['SETTLEMENT_DATE']

    if hour >= 24:
        hour -= 24
        datetime += timedelta(days=1)

    return datetime.replace(hour=hour, minute=minute)

# Apply the function to each row
df['DATETIME'] = df.apply(calculate_datetime, axis=1)

### Weather Data

In [6]:
df_weather = pd.read_csv('data/external/historical_weather_bulk.csv')

df_w_london = df_weather[df_weather['lat'] == 51.503655][['dt_iso', 'temp', 'temp_min', 'temp_max', 'humidity']]
df_w_bath = df_weather[df_weather['lat'] == 51.387535][['dt_iso', 'temp', 'temp_min', 'temp_max', 'humidity']]
df_w_liverpool = df_weather[df_weather['lat'] == 53.402859][['dt_iso', 'temp', 'temp_min', 'temp_max', 'humidity']]

df_w_london.columns = ['dt_iso', 'temp_london', 'temp_min_london', 'temp_max_london', 'humidity_london']
df_w_bath.columns = ['dt_iso', 'temp_bath', 'temp_min_bath', 'temp_max_bath', 'humidity_bath']
df_w_liverpool.columns = ['dt_iso', 'temp_liverpool', 'temp_min_liverpool', 'temp_max_liverpool', 'humidity_liverpool']

combined_df = df_w_london.merge(df_w_bath, on='dt_iso', how='outer')
combined_df = combined_df.merge(df_w_liverpool, on='dt_iso', how='outer')

combined_df = combined_df[['dt_iso', 'temp_london', 'temp_bath', 'temp_liverpool', 'temp_min_london', 'temp_min_bath', 'temp_min_liverpool', 
                           'temp_max_london','temp_max_bath', 'temp_max_liverpool', 'humidity_london', 'humidity_bath', 'humidity_liverpool']]

Convert ISO time to the timestamp format used above

In [7]:
date_format = "%Y-%m-%d %H:%M:%S +0000 UTC"
combined_df['dt_iso'] = pd.to_datetime(combined_df['dt_iso'], format=date_format).dt.strftime('%Y-%m-%d %H:%M:%S')
combined_df['DATETIME'] = combined_df['dt_iso']
combined_df = combined_df.drop(columns=['dt_iso'])

Merge with weather data, keep in mind that this reduces frequency to hourly due to the weather dataset

In [8]:
combined_df['DATETIME'] = pd.to_datetime(combined_df['DATETIME'])
df['DATETIME'] = pd.to_datetime(df['DATETIME'])
merged_df = pd.merge(df, combined_df, on='DATETIME', how='inner')

merged_df

,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,temp_liverpool,temp_min_london,temp_min_bath,temp_min_liverpool,temp_max_london,temp_max_bath,temp_max_liverpool,humidity_london,humidity_bath,humidity_liverpool
0,2023-01-02,1,21043,24680,18995,2815,6545,0,14284,0,...,278.79,279.95,279.03,277.96,281.17,281.56,279.61,93,93,93
1,2023-01-02,3,21348,25269,19327,2773,6545,0,14284,0,...,278.81,279.36,279.58,278.06,281.66,281.14,279.38,92,94,94
2,2023-01-02,5,19781,24135,17806,2712,6545,0,14284,0,...,278.75,279.36,279.58,278.06,281.12,281.56,279.38,92,94,94
3,2023-01-02,7,18511,22969,16459,2615,6545,0,14284,0,...,278.61,278.81,279.66,277.45,280.55,281.69,279.11,93,94,94
4,2023-01-02,9,17363,21875,15221,2511,6545,0,14284,0,...,278.75,278.25,279.66,277.45,280.55,281.56,279.21,93,94,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7850,2023-11-16,39,37489,39082,33728,768,6488,0,15595,0,...,280.29,279.95,277.03,278.94,281.66,281.04,281.51,88,89,90
7851,2023-11-16,41,35214,36802,31696,625,6488,0,15595,0,...,280.23,279.95,277.03,279.12,281.66,281.01,280.70,87,91,90
7852,2023-11-16,43,32697,34287,29445,520,6488,0,15595,0,...,280.09,279.39,277.03,278.83,281.17,281.01,280.60,87,92,91
7853,2023-11-16,45,29899,31488,26916,447,6488,0,15595,0,...,279.95,277.72,278.12,279.12,282.78,281.14,280.59,84,93,93


### Create Unlagged Target Variable

In [9]:
# Create an unlagged target variable and a lagged feature (yesterday's national demand)
merged_df['ND_TARGET'] = merged_df['ND'].shift(-1)
merged_df = merged_df.drop(merged_df.tail(1).index)
merged_df['ND_TARGET'] = merged_df['ND_TARGET'].astype(int)
merged_df['ND_PREV'] = merged_df['ND']
merged_df = merged_df.drop(columns=['ND'])

### Export

In [10]:
df_full = merged_df[['DATETIME', 'ND_TARGET', 'SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND_PREV', 'TSD',
       'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION',
       'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION',
       'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING',
       'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW',
       'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'month', 'day_of_week',
       'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos',
       'settlement_period_sin', 'settlement_period_cos',  'temp_london', 
       'temp_bath', 'temp_liverpool', 'temp_min_london',
       'temp_min_bath', 'temp_min_liverpool', 'temp_max_london',
       'temp_max_bath', 'temp_max_liverpool', 'humidity_london',
       'humidity_bath', 'humidity_liverpool']]

df_train = merged_df[['DATETIME', 'ND_TARGET','ND_PREV', 'TSD',
       'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION',
       'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION',
       'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING',
       'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW',
       'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW',
       'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos',
       'settlement_period_sin', 'settlement_period_cos', 'temp_london', 
       'temp_bath', 'temp_liverpool', 'temp_min_london',
       'temp_min_bath', 'temp_min_liverpool', 'temp_max_london',
       'temp_max_bath', 'temp_max_liverpool', 'humidity_london',
       'humidity_bath', 'humidity_liverpool']]

df_full.to_csv('data/custom/df_full.csv', index=False)
df_train.to_csv('data/custom/df_train.csv', index=False)